# Doppler Transform
This notebook illustrates the Doppler functional transform. This wideband Doppler effect operates over the entire sampled band by rescaling the input time series for a constant relative velocity and propagation speed.

In [ ]:
from torchsig.signals.signal_types import Signal
from torchsig.datasets.dataset_metadata import DatasetMetadata
from torchsig.signals.builders.tone import ToneSignalBuilder
import torchsig.transforms.functional as F
import torchsig.utils.dsp as dsp
from torchsig.utils.dsp import (
    TorchSigComplexDataType
)

import numpy as np
from scipy.signal import find_peaks
%matplotlib inline
import matplotlib.pyplot as plt
from scipy.constants import c

Function for generating the input constant tone test signal.

In [ ]:
def generate_tone_signal(num_iq_samples: int = 10, scale: float = 1.0) -> Signal:
    """Generate a scaled, high SNR baseband tone Signal.

        Args:
        num_iq_samples (int, optional): Length of sample. Defaults to 10.
        scale (int, optional): scale normalized signal data. Defaults to 1.0.

        Returns:
            signal: generated Signal.

    """
    sample_rate = 10e6
    md = DatasetMetadata(
        num_iq_samples_dataset = num_iq_samples,
        fft_size = 4,
        impairment_level = 0,
        sample_rate = sample_rate,
        num_signals_max = 1,
        num_signals_min = 1,
        num_signals_distribution = [1.0],
        snr_db_min = 100.0,
        snr_db_max = 100.0,
        signal_duration_min = 1.00*num_iq_samples/sample_rate,
        signal_duration_max = 1.00*num_iq_samples/sample_rate,
        signal_bandwidth_min = sample_rate/4,
        signal_bandwidth_max = sample_rate/4,
        signal_center_freq_min = 0.0,
        signal_center_freq_max = 0.0,         
        class_list = ['tone'],
        class_distribution = [1.0],
        seed = 42
    )

    builder = ToneSignalBuilder(
        dataset_metadata = md, 
        class_name = 'tone',
        seed = 42
    )
    signal = builder.build()

    # normalize, then scale data   
    signal.data = F.normalize(
        data = signal.data,
        norm_order = 2,
        flatten = False
    )
    signal.data = np.multiply(signal.data, scale).astype(TorchSigComplexDataType)

    return signal

Place the input tone signal frequency at 0.2, then apply the wideband Doppler effect with a time scaling factor of 1.03. Estimate the frequency shift and compare it with the true shift.

In [ ]:
rng = np.random.default_rng(42)

fc = 0.2   # tone center frequency
N = 10000  # tone signal samples
sampling_rate = 4.0

# input tone signal
tone_bb_data = generate_tone_signal(num_iq_samples = N, scale = 1.0).data 
tone_data = tone_bb_data * np.exp(2j * np.pi * fc * np.arange(N) / sampling_rate) 

# Doppler parameters
velocity = -1e7
propagation_speed = 2.9979e8
alpha = propagation_speed / (propagation_speed - velocity)

# apply Doppler transform
data_d = F.doppler(
    data = tone_data,
    velocity = velocity,
    propagation_speed = propagation_speed,
)
print(f"Doppler alpha factor: {alpha:.4f}")
print(f"True tone frequency pre-Doppler: {fc:.4f}")
print(f"True tone frequency post-Doppler: {fc*alpha:.4f}")

# estimate frequency shift
D = np.abs(np.fft.fft(data_d, norm='ortho'))
freqs = np.fft.fftfreq(len(D)) * sampling_rate
peaks, _ = find_peaks(D, height=0.1, distance=100)
print(f"Estimated tone frequency post-Doppler: {freqs[peaks[0]]:.4f}")

Plot the input and output spectrum magnitudes. Note that the output magnitude frequency response is not ideally clean - it has some residual frequency spurious components from the multiphase polyphase resampler that implements the wideband Doppler shift.

In [ ]:
plt.style.use('dark_background')
fig, ax = plt.subplots()

spectrum, freqs, _ = ax.magnitude_spectrum(tone_data, Fs=sampling_rate, scale='linear', sides='twosided', color='white');
spectrum, freqs, _ = ax.magnitude_spectrum(data_d, Fs=sampling_rate, scale='linear', sides='twosided', color='red');
ax.set_yscale('log')
ax.set_ylabel('Magnitude');
plt.ylim([1e-15, None]);